This here notebook is going to be an alternative plate solve solution, in case AIJ is messing with us somehow...pretty sure this comes from

https://astroquery.readthedocs.io/en/latest/astrometry_net/astrometry_net.html

cyx (reanalysis): 

umf: The B files aren't work at all, even with a median filter. Try just R? The way I solved this was to crank up the theshhold to like 800, and give the time also 600 (s) [ note that the timeout is not set correctly on the sheet, and needs to be set correctly in the .config file!] . That reduced the time enough for it to work...and it really took like 5-10 min per image! Yeah had to run this several times with different threshold values to get everything. Now working on B, and it's going, but slowly...

sse: (R) Worked at thresh 75 and 25. (B) Did them all straight up!

koy: Can't get the U to solve, even with the median filter on...my notes say this used to work! Ugh looks like some astronmetery.net problem, works now!

In [1]:
# RA and Dec guess, makes it go faster
RA_Guess=326.833497955
Dec_Guess=-26.5344445089
rad=1 #arcsec? No, I think this is degrees by looking at the source!
stack_dir="../WorkingWCS" # which images will we be doing WCS on?
skip_flag=0 #0 skips if the WCS exists, Change to 1 if you want to overwrite the existing WCS. 
#Note this flag DOES skip if the WCS was generated with this script!
median_filter_flag=1 # 0 does no median filter, 1 does the filter, maybe needed for U filter.
# This next line changes the threshold so that the number of sources found are fewer...try like 800 to ssrsy do that!
# Good defaults seem to be in the 50-100 range.
thresh = 25
# the next line changes the timeout
timeo = 600

In [2]:
from astropy.table import Table
from astroquery.astrometry_net import AstrometryNet
import os  #Call commands from outside Python
from astropy.io import fits #FITS files handling
import numpy as np
from pathlib import Path # to remove files

These config files set things that can't get changed later - like trying to use the timeout command on the CLI, just defaults to whatever is written in the stupid file! But even 4 minutes is sometimes not enough, so try to get the scales set right above!

In [3]:
#from astropy.config import get_config_dir

#get_config_dir()  

In [4]:
#from astropy.config import reload_config

#reload_config()

In [5]:
# function to remove the AIJ-added WCS (probably doesn't work for WCS added in other ways!)
def clean_WCS(header):
    for i in range(len(header)):
        if header[i]==' WCS created by AIJ link to Astronomy.net website': # delete everything after this!
            print("Erasing exising WCS headers.")
            for j in range(i,len(header)):
                del header[i]
            return

In [6]:
# function to glue WCS headers back in
def glue_WCS(header,wcs_header):
    for i in range(len(wcs_header)):
        header.append(wcs_header)

In [7]:
ast = AstrometryNet()
#ast.api_key = 'nscnpfezrirsdsum'
ast.api_key = 'goiagdyjiupsvvzh'
#ast.key='nscnpfezrirsdsum'

In [8]:
image_array=[]
for f in os.listdir(stack_dir): # create list of raw images
    image_array.append(f)
    
image_array.sort() # sorted by name so that discovery is first.
image_array

['SUM_Aligned_20230616-I.fits',
 'SUM_Aligned_20230616-U.fits',
 'SUM_Aligned_20230624-R.fits',
 'SUM_Aligned_20230624-U.fits',
 'SUM_Aligned_20230625-R.fits',
 'SUM_Aligned_20230625-U.fits',
 'SUM_Aligned_20230626-R.fits',
 'SUM_Aligned_20230626-U.fits',
 'SUM_Aligned_20230711-R.fits',
 'SUM_Aligned_20230711-U.fits',
 'SUM_Aligned_20230712-R.fits',
 'SUM_Aligned_20230712-U.fits',
 'SUM_Aligned_20230713-R.fits',
 'SUM_Aligned_20230713-U.fits',
 'SUM_Aligned_20230714-R.fits',
 'SUM_Aligned_20230714-U.fits',
 'SUM_Aligned_20230715-R.fits',
 'SUM_Aligned_20230715-U.fits',
 'SUM_Aligned_20230716-R.fits',
 'SUM_Aligned_20230716-U.fits',
 'SUM_Aligned_20230720-R.fits',
 'SUM_Aligned_20230720-U.fits',
 'SUM_Aligned_20230722-R.fits',
 'SUM_Aligned_20230722-U.fits',
 'SUM_Aligned_20230723-R.fits',
 'SUM_Aligned_20230723-U.fits',
 'SUM_Aligned_20230724-R.fits',
 'SUM_Aligned_20230724-U.fits',
 'SUM_Aligned_20230725-R.fits',
 'SUM_Aligned_20230725-U.fits',
 'SUM_Aligned_20230726-R.fits',
 'SUM_Al

In [9]:
#file_wo_WCS_name=stack_dir+"/"+image_array[3]
#file_wo_WCS = fits.open(file_wo_WCS_name)
#file_wo_WCS[0].header

In [10]:
#clean_WCS(file_wo_WCS[0].header)

In [11]:
#file_wo_WCS[0].header

In [12]:
# clean the headers anyway!
# ohhhh tihs does NOT clean the headers if I used this script to generate the WCS!
for i in range(len(image_array)):
    file_wo_WCS_name=stack_dir+"/"+image_array[i]
    file_wo_WCS = fits.open(file_wo_WCS_name)
    clean_WCS(file_wo_WCS[0].header)
    hdu = fits.PrimaryHDU(file_wo_WCS[0].data,file_wo_WCS[0].header)
    hdu.writeto(file_wo_WCS_name,overwrite=True)

In [13]:
#ast.show_allowed_settings()

Ok, for the U filter, AIJ can do it *as long as it's median filtered 2x2!!* So we need to do that - maybe with just a catalog, median filtered, and pass with solve_from_catalog.

Do I have to write my own damn median filter?!

In [14]:
#file_wo_WCS[0].data

In [15]:
#file_wo_WCS[0].data.shape[0]

In [16]:
# hard coding radius of 2 right nowm, since that's all we needed for our U filter in koy!
def median_filter(data):
    filtered=file_wo_WCS[0].data
    for i in range(1,data.shape[0]-1):
        for j in range(1,data.shape[0]-1):
            pixels=[data[i-1][j-1],data[i][j-1],data[i][j],data[i+1][j],data[i][j+1]]
            #print(pixels,np.median(pixels))
            filtered[i][j]=np.median(pixels)
    return filtered

In [17]:
#median_filtered_data=median_filter(file_wo_WCS[0].data)

In [18]:
#median_file_name=stack_dir+"/"+"med.fits"
#file_wo_WCS_name=stack_dir+"/"+image_array[i]
#file_wo_WCS = fits.open(file_wo_WCS_name)
#median_hdu = fits.PrimaryHDU(median_filtered_data,file_wo_WCS[0].header)
#median_hdu.writeto(median_file_name,overwrite=True)

In [19]:
%%time
skip=0
good =0
bad=0
for i in range(len(image_array)):
    file_wo_WCS_name=stack_dir+"/"+image_array[i]
    print("Processing ",i," of ", len(image_array), " : ", file_wo_WCS_name)
    file_wo_WCS = fits.open(file_wo_WCS_name)
    if ( skip_flag==1 ): # if skip flag is 1, overwrite the WCS no matter what
        #median filter part!
            if median_filter_flag==1:
                median_filtered_data=median_filter(file_wo_WCS[0].data)
                print("Performed median filter...")
                median_file_name=stack_dir+"/"+"med.fits"
                median_hdu = fits.PrimaryHDU(median_filtered_data,file_wo_WCS[0].header)
                median_hdu.writeto(median_file_name,overwrite=True)
                wcs_header = ast.solve_from_image(median_file_name, solve_timeout=timeo,center_ra=RA_Guess,center_dec=Dec_Guess,radius=rad,detect_threshold=thresh)
            else:
                #wcs_header = ast.solve_from_image(file_wo_WCS_name, solve_timeout=240,center_ra=RA_Guess,center_dec=Dec_Guess,radius=rad)
                # This version has even more speedup to it:
                #wcs_header = ast.solve_from_image(file_wo_WCS_name, center_ra=RA_Guess,center_dec=Dec_Guess,radius=rad,scale_units='degwidth',scale_type='ul',scale_lower=0.1, scale_upper=5.,detect_threshold=thresh)
                wcs_header = ast.solve_from_image(file_wo_WCS_name, solve_timeout=timeo,center_ra=RA_Guess,center_dec=Dec_Guess,radius=rad,detect_threshold=thresh)
            hdu_w_WCS = fits.PrimaryHDU(file_wo_WCS[0].data,file_wo_WCS[0].header+wcs_header)
            hdu_w_WCS.writeto(file_wo_WCS_name,overwrite=True)
            print("\n\nFile done",file_wo_WCS_name)
            good=good+1
    # if skip flag is zero, check for the existance before solving.
    else:
        try:
            file_wo_WCS[0].header['WCSAXES']
            print("WCS already exists for",file_wo_WCS_name,"skipping...\n")
            skip = skip +1
        except:
            try: 
                # remove WCS keywords if they exist
                #clean_WCS(file_wo_WCS[0].header)
            
                #median filter part!
                if median_filter_flag==1:
                    print("Median Filter requested....")
                    median_filtered_data=median_filter(file_wo_WCS[0].data)
                    print("Median filtering finished...")
                    median_file_name=stack_dir+"/"+"med.fits"
                    median_hdu = fits.PrimaryHDU(median_filtered_data,file_wo_WCS[0].header)
                    median_hdu.writeto(median_file_name,overwrite=True)
                    wcs_header = ast.solve_from_image(median_file_name, solve_timeout=timeo,center_ra=RA_Guess,center_dec=Dec_Guess,radius=rad,detect_threshold=thresh)
                    if wcs_header:
                        print("WCS Found!",wcs_header)
                    else:
                        print("WCS failed for unknown reason...")
                    # this line borked it up, for some reason!
                    #Path.unlink(median_file) # doens't get hit if there is an unknown error!
                else:
                    print("Finding WCS...")
                    wcs_header = ast.solve_from_image(file_wo_WCS_name, solve_timeout=timeo,center_ra=RA_Guess,center_dec=Dec_Guess,radius=rad,detect_threshold=thresh)
                    print("WCS Found!",wcs_header)
                hdu_w_WCS = fits.PrimaryHDU(file_wo_WCS[0].data,file_wo_WCS[0].header+wcs_header)
                hdu_w_WCS.writeto(file_wo_WCS_name,overwrite=True)
                print("\n\nFile done",file_wo_WCS_name)
                good=good+1
            except:
                print("\n\nUnknown Error on ", file_wo_WCS_name,"...continuing...\n")
                bad=bad+1

#remove the median file if you created it
if os.path.exists(median_file_name):
    os.remove(median_file_name)
 
print("\n\n\nGood: ", good, "bad: ",bad,"skip:", skip)

Processing  0  of  38  :  ../WorkingWCS/SUM_Aligned_20230616-I.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20230616-I.fits skipping...

Processing  1  of  38  :  ../WorkingWCS/SUM_Aligned_20230616-U.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20230616-U.fits skipping...

Processing  2  of  38  :  ../WorkingWCS/SUM_Aligned_20230624-R.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20230624-R.fits skipping...

Processing  3  of  38  :  ../WorkingWCS/SUM_Aligned_20230624-U.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20230624-U.fits skipping...

Processing  4  of  38  :  ../WorkingWCS/SUM_Aligned_20230625-R.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20230625-R.fits skipping...

Processing  5  of  38  :  ../WorkingWCS/SUM_Aligned_20230625-U.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20230625-U.fits skipping...

Processing  6  of  38  :  ../WorkingWCS/SUM_Aligned_20230626-R.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_2023062

Set MJD-AVG to 60153.655956 from DATE-AVG'. [astropy.wcs.wcs]


Finding sources




Unknown Error on  ../WorkingWCS/SUM_Aligned_20230728-R.fits ...continuing...

Processing  35  of  38  :  ../WorkingWCS/SUM_Aligned_20230728-U.fits
Median Filter requested....
Median filtering finished...
Determining background stats


Set MJD-AVG to 60153.638335 from DATE-AVG'. [astropy.wcs.wcs]


Finding sources


Unknown Error on  ../WorkingWCS/SUM_Aligned_20230728-U.fits ...continuing...

Processing  36  of  38  :  ../WorkingWCS/SUM_Aligned_20240618-R.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20240618-R.fits skipping...

Processing  37  of  38  :  ../WorkingWCS/SUM_Aligned_20240618-U.fits
WCS already exists for ../WorkingWCS/SUM_Aligned_20240618-U.fits skipping...




Good:  0 bad:  2 skip: 36
CPU times: user 2min 38s, sys: 118 ms, total: 2min 38s
Wall time: 2min 38s
